In [ ]:
##import
import pandas as pd
import numpy as np
from IPython.core.display import display
from tqdm import tqdm_notebook as tqdm
from copy import deepcopy as cp

##Visualization
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode()

##visualization
from ipywidgets import interact
#from bokeh import mpl
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import gridplot
from bokeh.palettes import Category10 as palette
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
import itertools

##import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, auc
from sklearn.feature_selection import SelectKBest, f_classif, RFECV

import boruta_py

## statistical visualization
from string import ascii_letters
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Kozuka Gothic Pro'
%matplotlib inline

## はじめに
前回の記事で、変数選択(Feature Selection)についてまとめたので、実際に実装してみます。目的を見失うのを防ぐために、何が目的でどんなことをするのか実験設定を明記します。

### 実験設定
#### 目的
いくつかの変数選択手法によって変数を選択し、モデルの改善を確かめる。
#### 用いるデータ
irisとかもう見すぎて飽きてるのでKaggleから取ってきました。
https://www.kaggle.com/mlg-ulb/creditcardfraud/kernels

このデータは、クレジットカードのデータから不正使用されたかどうか当てるタスクに用いることができます。ただしオリジナルデータは機密情報で会社としては公開できないので、このデータはオリジナルのデータをPCAしたものになります。説明変数の実態がなんなのかわからないのでハンドクラフトに特徴をピックアップすることができず、今回の変数選択手法を試すのにはぴったりなデータセットだと思います。ただし、Classが1であるサンプルが著しく少ないインバランスなデータです。中身を見てみるとこんな感じです。

In [ ]:
##acquire data
df = pd.read_csv('./creditcard.csv')
df.head()

#### 用いる変数選択手法
本記事のメインディッシュです。
* Filter Method
* Wrapper Method
    * sklearn
    * Boruta
**(あとでもっと詳しく書こう)**

#### 用いる判別器
ロジスティック回帰を用いて行います。理由としては、計算が軽量であることや線形分離で判別できそうということが挙げられます。(あとで可視化します。)
#### 評価指標
10CVでRP-AUC(PR曲線の下側の面積)の標本平均を用います。PR曲線はROC曲線の親戚のようなもので、インバランスデータを評価するのに適しています。詳しくは過去の記事を見てください。
** ULR挿入 **

#### 行わないこと
* インバランスへの対応(用いるデータはClass==1が著しく少ないデータになっています。)
* パラメーターサーチ
* このデータに対してどの判別器が適しているのか
* クラス分類をするための閾値の設定
* この判別に対して最終的なモデルを示す

つまり、変数選択の結果、判定器が改善されたかだけを見ます。

## データを少し見てみる
datasetのV1~V3を可視化してみます。これだけでもなんとなく線形分離可能なんじゃないかと予想できます。

In [ ]:
df0 = df[df.Class == 0]
df1 = df[df.Class == 1]
##random under sampling
df0u = df0.sample(frac = 0.04)
## make trace
trace0 = go.Scatter3d(
    x = df0u.V1,
    y = df0u.V2,
    z = df0u.V3,
    name = 'class0',
    mode = 'markers',
    #opacity = 0.4,
    marker = dict(
        size = 3
    )
)
trace1 = go.Scatter3d(
    x = df1.V1,
    y = df1.V2,
    z = df1.V3,
    name = 'class1',
    mode = 'markers',
    marker = dict(
        size = 3
    )
)
## concatnate traces
data = [trace0, trace1]

## define layout
layout = go.Layout(
    title='3D-PCA',
    width=700,
    height=600,
    scene = dict(
        xaxis = dict(
            nticks=4, range = [min(df.V1),max(df.V1)], title='V1'),
        yaxis = dict(
            nticks=4, range = [min(df.V2),max(df.V2)], title='V2'),
        zaxis = dict(
            nticks=4, range = [min(df.V3),max(df.V3)], title='V3')
    ),
    showlegend=True)

fig = dict(data=data, layout=layout)
offline.iplot(fig)

## すべての特徴を用いた場合
変数選択が有用ということを示すには、比較対象が必要です。すべての特徴を用いてロジスティック回帰を行った場合に、どれぐらいのスコアが出るのか確認してから変数選択した場合と比較しましょう。

In [ ]:
##make matrix
X = df.drop('Class', axis=1)
y = df.Class

def print_pr_auc_score(X, y):
    ##10-foldCV, LogisticRegression, PR_AUC
    pr_auc = cross_val_score(LogisticRegression(), X, y, scoring="average_precision", cv=10)
    print('各分割でのスコア:',pr_auc)
    print('\nその平均:',np.mean(pr_auc))

print_pr_auc_score(X, y)

すべての何も考えず特徴を用いたとき、**0.763**となりました。これを基準にします。

## Filter Method
まずFilter Methodによる変数選択を行ってみます。今回、特徴(説明変数)が連続値で、目的変数がカテゴリーなので、**前回の記事**の表に従うとLDAを用いて、目的変数に対して特徴が効いてるのか見ることになります。がしかし。sklearnで楽にやりたかったこともあり今回はANOVAのF値を指標にしました。これは、sklearn.feature_selection.SelectKBestでf_classif(判別分析用)を指定したときのスコアになります。
ANOVAのF値については、いろいろ検索してみましたがこちらが私的にわかりやすいと感じました。(このF値とF分布を用いると検定の枠組みに持っていくことができて、その特徴がどの程度の確率で有意なのかも定量的に判断することができますが今回は考えないことにします。)http://www.ipc.shimane-u.ac.jp/food/kobayasi/anova.htm

さて、では実際に変数選択してみましょう。まずは各クラスごとに各特徴の確率分布を描くことによって、効いてそうな特徴を目視で選択してみることにします。

### 目視により選択
すべての説明変数の確率分布を表示してみました。図が多いのでここではわかりやすい図だけ示します。

In [ ]:
%matplotlib inline
for i in tqdm(range(len(df.columns)-1)):
    g = sns.distplot(df0.iloc[:,i], color='green')
    g = sns.distplot(df1.iloc[:,i], color='red') 
    plt.show()

線形分離でうまく両分布が分かれそうな特徴を選ぶと、V3, V4, V10, V11, V12, V14, V16となりました。これを使ってロジスティック回帰を評価して見ようと思います。

In [ ]:
##make matrix
X = df[['V3','V4','V10','V11','V12','V14','V16']]
y = df.Class

##10-foldCV, LogisticRegression, PR_AUC
pr_auc = cross_val_score(LogisticRegression(), X, y, scoring="average_precision", cv=10)
print('各分割でのスコア:',pr_auc)
print('\nその平均:',np.mean(pr_auc))

目視で変数選択したときのスコアは**0.782**となりました。

### sklearn.feature_selection.SelectKBestによる選択
目視ではなくもっと機械的に決めます。前述したようにANOVAのF値を用いています。その上位K個を返すといった関数です。
ただし、いくつ変数が選ばれたらモデルとして良いのかわからないため、選ぶ変数の数を探索します。

In [ ]:
##make matrix
X = df.drop('Class', axis=1)
y = df.Class

scores=[]
for n in tqdm(range(1,len(X.columns))):
    print('\n説明変数の数n=',n)
    ##select features
    select = SelectKBest(k=n)
    select.fit(X, y)
    mask = select.get_support()
    X_selected = X.iloc[:,mask]
    ##10-foldCV, LogisticRegression, PR_AUC
    pr_auc = cross_val_score(LogisticRegression(), X_selected, y, scoring="average_precision", cv=10)
    scores.append(np.mean(pr_auc))    
    print('平均のPR_AUC:',scores[n-1])

    ## visualization
    plt.matshow(mask.reshape(1, -1), cmap='gray_r')
    plt.tick_params(labelleft = 'off')
    plt.xlabel('使われた特徴. 黒が選択されたもの', fontsize=15)
    plt.show()
    

モデルに使う特徴の数とスコアの関係を図示します。

In [ ]:
p = figure(
    title = "n vs. PR_AUC", 
    plot_width=500, plot_height=500,
)
p.line(
    range(1,len(scores)+1),
    scores
)
show(p)

21番目の説明変数を加えた途端スコアが大きく伸びました。21番目に追加された説明変数'Time'の確率分布を見てみましょう。

In [ ]:
%matplotlib inline
sns.distplot(df0.iloc[:,21], color='green')
sns.distplot(df1.iloc[:,21], color='red')
plt.xlim(-5, 5)
plt.show()

分けられるか微妙ですが、もしかしたら他の変数に対するマルチコが少なく他の変数と組み合わせると有効に働くのかも知れません。上位5つの特徴だけを用いたときと、上位5つ＋'Time'を用いたときを比較しましょう。

In [ ]:
##make matrix
X = df.drop('Class', axis=1)
y = df.Class

##select features
select = SelectKBest(k=5)
select.fit(X, y)
mask = select.get_support()
X_selected = pd.concat([df.Time, X.iloc[:,mask]], axis=1)
##10-foldCV, LogisticRegression, PR_AUC
pr_auc = cross_val_score(LogisticRegression(), X_selected, y, scoring="average_precision", cv=10)
print('上位5つのみのとき:',scores[4])
print('上位5つ＋\'Time\'のとき:', np.mean(pr_auc) )

Timeも特徴に含めたほうが、結果として一番良いスコアが出ました。SelectKBestでは選ぶことができなかったので、FilterMethodを用いた結果としては、**0.782**の方を採用します。
Timeは有用な変数だったにもかかわらず、SelectKBestで見逃されたようです。多重共線性を考慮できないのがFilterMethodの問題点です。

## Wrapper Method
### sklearn.feature_selection.RFECVによる選択
recursive feature eliminationを用いた選択です。

In [ ]:
select = RFECV(LogisticRegression(), cv=10, scoring='average_precision')
select.fit(X, y)
mask = select.support_

## visualization
plt.matshow(mask.reshape(1, -1), cmap='gray_r')
plt.tick_params(labelleft = 'off')
plt.xlabel('使われた特徴. 黒が選択されたもの', fontsize=15)
plt.show()

選ばれた特徴でスコアを算出します。

In [ ]:
print('選ばれた変数の数',select.n_features_)
print('各変数のランキング',select.ranking_)
select.grid_scores_

In [ ]:
X_selected = X.iloc[:,mask]
##10-foldCV, LogisticRegression, PR_AUC
pr_auc = cross_val_score(LogisticRegression(), X_selected, y, scoring="average_precision", cv=10)
print('平均のPR_AUC:',np.mean(pr_auc))

先程と同じように'Time'が選ばれていません。問題となったTimeも加えて評価してみました。

In [ ]:
X_selected = pd.concat([df.Time, X.iloc[:,mask]], axis=1)
##10-foldCV, LogisticRegression, PR_AUC
pr_auc = cross_val_score(LogisticRegression(), X_selected, y, scoring="average_precision", cv=10)
print('\'Time\'を加えたのとき:', np.mean(pr_auc) )

少しだけスコアが上がりました。しかしこれもRFECVが自動的に選んではくれなかったものなので、ここでのスコアは**0.788**とします。

### Borutaによる変数選択
これはrandomForestを用いた変数選択手法の一つで、詳しいアルゴリズムはこちらの3ページに書いてあります。https://www.google.co.jp/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&ved=0ahUKEwiEzMP4p9naAhWBk5QKHbRjC9oQFggoMAA&url=https%3A%2F%2Fwww.jstatsoft.org%2Farticle%2Fview%2Fv036i11%2Fv36i11.pdf&usg=AOvVaw3tyiHN0BCe2fkkAA6xEVDE

In [ ]:
##make matrix
X = df.drop('Class', axis=1).values
y = df.Class.values

forest = RandomForestClassifier()
# define Boruta feature selection method
select = boruta_py.BorutaPy(forest, n_estimators=10)
select.fit(X, y)
mask = select.support_

## visualization
plt.matshow(mask.reshape(1, -1), cmap='gray_r')
plt.tick_params(labelleft = 'off')
plt.xlabel('使われた特徴. 黒が選択されたもの', fontsize=15)
plt.show()

In [ ]:
X_selected = df.iloc[:,mask]
##10-foldCV, LogisticRegression, PR_AUC
pr_auc = cross_val_score(LogisticRegression(), X_selected, y, scoring="average_precision", cv=10)
print('平均のPR_AUC:',np.mean(pr_auc))

**0.796**というスコアになりました。適当にパラメーターを決めたり、ロジスティック回帰でいいのかという疑問はありますが、ひとまずこれを採用したいと思います。また、計算時間に関しては、2.5GHzCorei5で4時間程度かかりました。計算中はメモリも2GBぐらい持っていかれたと思います。